In [1]:
from dask.distributed import Client
client = Client()

In [2]:
import dask.array as da
import pandas as pd
import numpy as np

https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data

This dataset comprises of grocery sales and other meta data around it like holiday schedule, transaction data etc. 

The total size of the dataset is 4.5Gb.

In [3]:
#grocery_data = pd.read_csv('Dask_grocery/train.csv')

In [4]:
from dask import dataframe as dd
dfd = dd.read_csv(
    'Dask_grocery/train.csv', 
    delimiter=',',
    blocksize=64000000 # = 64 Mb chunks
)

In [5]:
dfd.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


# More than 125 Million rows in the dataset

In [7]:
dfd.count()
dfd.compute()
#more than 125 Million rows

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:49205 remote=tcp://127.0.0.1:65449>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:49207 remote=tcp://127.0.0.1:65449>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:49206 remote=tcp://127.0.0.1:65449>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:49209 remote=tcp://127.0.0.1:65449>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:49208 remote=tcp://127.0.0.1:65449>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:49213 remote=tcp://127.0.0.1:65449>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:49215 remote=tcp://127.0.0.1:65449>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:49217 remote=tcp://127.0.0.1:65449>
distribu

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN
...,...,...,...,...,...,...
130710,125497035,2017-08-15,54,2089339,4.0,0.0
130711,125497036,2017-08-15,54,2106464,1.0,1.0
130712,125497037,2017-08-15,54,2110456,192.0,0.0
130713,125497038,2017-08-15,54,2113914,198.0,1.0


In [ ]:
from dask import dataframe as dd
stores = dd.read_csv(
    'Dask_grocery/stores.csv', 
    delimiter=',',
    blocksize=64000000 # = 64 Mb chunks
)

In [ ]:
stores.head()

In [ ]:
dfd_stores = dfd.join(stores.set_index('store_nbr'), on='store_nbr',how = 'inner')

In [ ]:
dfd_stores.head()

# Best performing Store with respect to unit sales

In [ ]:


dfd_UnitSales = dfd_stores.groupby(['store_nbr'])['unit_sales'].sum().rename('sums').reset_index()

type(dfd_UnitSales)
dfd_UnitSales.sums.nlargest(5).compute()


# Insight 2: Store number 37 has highest sales during the given time period with more than 6.2 million units sold.

# ============================================================

# Impact of promotions on the sale of items?

Around 16% of total items were on sale

In [26]:
#highest promotional item sale
dfd1 = dfd['onpromotion'] >= 0.0
dfd_fil = dfd[dfd1]

dfd_itemSales1 = dfd_fil.groupby('item_nbr')['unit_sales'].mean().rename('sums').reset_index()

dfd_itemSales1.sums.nlargest(5).compute()

3543    6743.809280
1922    1135.322505
1501     229.648377
2449     226.545985
2264     113.885663
Name: sums, dtype: float64

In [22]:
# #highest promotional item sale
# # dfd1 = dfd['onpromotion'] >= 0.0
# # dfd_fil = dfd[dfd1]

# dfd_itemSales1 = dfd.groupby(['item_nbr','onpromotion'])['unit_sales'].sum().rename('sums').reset_index()

# dfd_itemSales1.compute()

In [18]:
dfd['onpromotion'].unique().compute()

0    NaN
1    0.0
2    1.0
Name: onpromotion, dtype: float64

In [27]:
#Average non promotional item sale
dfd1 = dfd['onpromotion'].isnull()
dfd_fil = dfd[dfd1]

dfd_itemSales1 = dfd_fil.groupby('item_nbr')['unit_sales'].mean().rename('sums').reset_index()

dfd_itemSales1.sums.nlargest(5).compute()

1962    322.900333
2392    251.196960
1523    245.707007
710     229.589389
2208    194.907563
Name: sums, dtype: float64

# Insight 2: Sales and promotion on items are weakly corelated average sales varying for some case only 

# ============================================================

#  Day of the week Analysis for best sale day?


In [5]:
import dask.dataframe as dd

dfd['my_dates'] = dd.to_datetime(dfd['date'])

dfd['day_of_week'] = dfd['my_dates'].dt.day_name()

In [6]:
dfd.compute()

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:50894 remote=tcp://127.0.0.1:50780>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:50895 remote=tcp://127.0.0.1:50780>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:50896 remote=tcp://127.0.0.1:50780>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:50898 remote=tcp://127.0.0.1:50780>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:50900 remote=tcp://127.0.0.1:50780>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:50899 remote=tcp://127.0.0.1:50780>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:50901 remote=tcp://127.0.0.1:50780>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:50905 remote=tcp://127.0.0.1:50780>
distribu

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,my_dates,day_of_week
0,0,2013-01-01,25,103665,7.0,NaN,2013-01-01,Tuesday
1,1,2013-01-01,25,105574,1.0,NaN,2013-01-01,Tuesday
2,2,2013-01-01,25,105575,2.0,NaN,2013-01-01,Tuesday
3,3,2013-01-01,25,108079,1.0,NaN,2013-01-01,Tuesday
4,4,2013-01-01,25,108701,1.0,NaN,2013-01-01,Tuesday
...,...,...,...,...,...,...,...,...
130710,125497035,2017-08-15,54,2089339,4.0,0.0,2017-08-15,Tuesday
130711,125497036,2017-08-15,54,2106464,1.0,1.0,2017-08-15,Tuesday
130712,125497037,2017-08-15,54,2110456,192.0,0.0,2017-08-15,Tuesday
130713,125497038,2017-08-15,54,2113914,198.0,1.0,2017-08-15,Tuesday


In [7]:
#Day of the week analysis

dfd_dow = dfd.groupby('day_of_week')['unit_sales'].mean().rename('avgSaleDOW').reset_index()

dfd_dow.avgSaleDOW.nlargest(7).compute()

3    10.596579
2     9.744609
1     8.312778
6     8.126039
0     7.944683
5     7.807375
4     7.085600
Name: avgSaleDOW, dtype: float64

# Thursdays have the highest sale during the week